In [7]:
%load_ext jupyter_spaces
from IPython.display import display

The jupyter_spaces extension is already loaded. To reload it, use:
  %reload_ext jupyter_spaces


In [21]:
import sympy as sym
from sympy import symbols, simplify
from sympy.vector import CoordSys3D, matrix_to_vector

S = CoordSys3D('S')

M, tet1, tet2 = symbols("M theta1 theta2", real=True)
mx1, my1, mz1 = symbols("m_x1 m_y1 m_z1", real=False)
mx2, my2, mz2 = symbols("m_x2 m_y2 m_z2", real=False)
J, mu0, g, t, A = symbols("J mu0 gamma t A", real=True, positive=True)
H = symbols("H", real=True)

m1 = mx1*S.i + my1*S.j + mz1*S.k
m2 = mx2*S.i + my2*S.j + mz2*S.k
M1_S = M*sym.cos(tet1)*S.i + M*sym.sin(tet1)*S.j
M2_S = M*sym.cos(tet1)*S.i + M*sym.sin(-tet1)*S.j

M1 = M1_S + m1
M2 = M2_S + m2

HZ = H*S.i


HD1_S = 0*S.k
HD1 = -mz1*S.k
HD2_S = 0*S.k
HD2 = -mz2*S.k


Hiec1_S = A*(M1_S*(M1_S.dot(M2_S)/(M**2)) - M2_S)
Hiec1 = A*(M1*(M1.dot(M2)/(M**2)) - M2)
Hiec2_S = A*(M2_S*(M2_S.dot(M1_S)/(M**2)) - M1_S)
Hiec2 = A*(M2*(M2.dot(M1)/(M**2)) - M1)


#Hiec1_S = A*(-M2_S)
#Hiec1_D = A*(-m2)
#Hiec1 = Hiec1_S+Hiec1_D

#Hiec2_S = A*(-M1_S)
#Hiec2_D = A*(-m1)
#Hiec2 = Hiec2_S+Hiec2_D

Heff1_S = HZ + HD1_S + Hiec1_S
Heff1 = HZ + HD1 + Hiec1

Heff2_S = HZ + HD2_S + Hiec2_S
Heff2 = HZ + HD2 + Hiec2

w = symbols("w", real=True)
jw = sym.I*w

dm1dt = jw*m1
dm2dt = jw*m2

eq1_S = g*mu0*MS1.cross(Heff1_S)
eq1 = dm1dt + g*mu0*(M1.cross(Heff1))
eq2_S = g*mu0*MS2.cross(Heff2_S)
eq2 = dm2dt + g*mu0*(M2.cross(Heff2))

eq_mat_S = eq2_S.to_matrix(S).row_insert(0, eq1_S.to_matrix(S))
eq_mat = eq2.to_matrix(S).row_insert(0, eq1.to_matrix(S))
#sol = sym.solve(M, m2.to_matrix(S).row_insert(0, m1.to_matrix(S)))

eq_mat_D = ((eq_mat-eq_mat_S)/(g*mu0)).expand()

to_comb_tmp = [mx1, my1, mz1, mx2, my2, mz2]
for i in range(6):
    for j in range(6):
        eq_mat_D = eq_mat_D.subs({to_comb_tmp[i]*to_comb_tmp[j]: 0})
        
eq_mat_D = sym.Matrix(eq_mat_D)
for i in range(6):
    eq_mat_D[i] = eq_mat_D[i].collect((mx1, my1, mz1, mx2, my2, mz2))

eq_mat_D

Matrix([
[                                 -A*M*m_z2*sin(theta1) + m_z1*(-A*M*sin(theta1) - M*sin(theta1)) + I*m_x1*w/(gamma*mu0)],
[                              A*M*m_z2*cos(theta1) + m_z1*(-A*M*cos(theta1) + H + M*cos(theta1)) + I*m_y1*w/(gamma*mu0)],
[ A*M*m_x1*sin(theta1) + A*M*m_x2*sin(theta1) - A*M*m_y2*cos(theta1) + m_y1*(A*M*cos(theta1) - H) + I*m_z1*w/(gamma*mu0)],
[                                   A*M*m_z1*sin(theta1) + m_z2*(A*M*sin(theta1) + M*sin(theta1)) + I*m_x2*w/(gamma*mu0)],
[                              A*M*m_z1*cos(theta1) + m_z2*(-A*M*cos(theta1) + H + M*cos(theta1)) + I*m_y2*w/(gamma*mu0)],
[-A*M*m_x1*sin(theta1) - A*M*m_x2*sin(theta1) - A*M*m_y1*cos(theta1) + m_y2*(A*M*cos(theta1) - H) + I*m_z2*w/(gamma*mu0)]])

In [22]:
lin_mat = sym.Matrix()

to_comb_tmp = [mx1, my1, mz1, mx2, my2, mz2]

for i in range(6):
    col = []
    for j in range(6):
         col.append(eq_mat_D[j].coeff(to_comb_tmp[i], 1))
    lin_mat = lin_mat.col_insert(i,sym.Matrix(col))

lin_mat = simplify(lin_mat)
lin_mat_det = lin_mat.det()


lin_mat_det0_sol = sym.solve(lin_mat_det, w,
          simplify=True, manual=False, implicit=False, 
          warn=True, check=True)

lin_mat

c:\users\sasha\appdata\local\programs\python\python38\lib\site-packages\sympy\solvers\solvers.py:1241: UserWarning: 
solution(s)                 can't be checked:
gamma*mu0*sqrt(4*A**2*M**2*cos(theta1)**2 - 4*A*H*M*cos(theta1) -
4*A*M**2*cos(theta1)**2 + 2*A*M**2 + H**2 + H*M*cos(theta1)),
-gamma*mu0*sqrt(H*(H + M*cos(theta1))), gamma*mu0*sqrt(H*(H +
M*cos(theta1))), -gamma*mu0*sqrt(4*A**2*M**2*cos(theta1)**2 -
4*A*H*M*cos(theta1) - 4*A*M**2*cos(theta1)**2 + 2*A*M**2 + H**2 +
H*M*cos(theta1))
  warnings.warn(filldedent("""


Matrix([
[ I*w/(gamma*mu0),                   0,               -M*(A + 1)*sin(theta1),                0,                   0,                     -A*M*sin(theta1)],
[               0,     I*w/(gamma*mu0), -A*M*cos(theta1) + H + M*cos(theta1),                0,                   0,                      A*M*cos(theta1)],
[ A*M*sin(theta1), A*M*cos(theta1) - H,                      I*w/(gamma*mu0),  A*M*sin(theta1),    -A*M*cos(theta1),                                    0],
[               0,                   0,                      A*M*sin(theta1),  I*w/(gamma*mu0),                   0,                M*(A + 1)*sin(theta1)],
[               0,                   0,                      A*M*cos(theta1),                0,     I*w/(gamma*mu0), -A*M*cos(theta1) + H + M*cos(theta1)],
[-A*M*sin(theta1),    -A*M*cos(theta1),                                    0, -A*M*sin(theta1), A*M*cos(theta1) - H,                      I*w/(gamma*mu0)]])

In [23]:
for cur_sol in lin_mat_det0_sol:
    display(simplify(cur_sol))

0

-gamma*mu0*sqrt(H*(H + M*cos(theta1)))

gamma*mu0*sqrt(H*(H + M*cos(theta1)))

-gamma*mu0*sqrt(4*A**2*M**2*cos(theta1)**2 - 4*A*H*M*cos(theta1) - 4*A*M**2*cos(theta1)**2 + 2*A*M**2 + H**2 + H*M*cos(theta1))

gamma*mu0*sqrt(4*A**2*M**2*cos(theta1)**2 - 4*A*H*M*cos(theta1) - 4*A*M**2*cos(theta1)**2 + 2*A*M**2 + H**2 + H*M*cos(theta1))

In [30]:
lin_mat_sol = sym.solve_linear_system(lin_mat.col_insert(6, sym.Matrix([0]*6)).subs({w: lin_mat_det0_sol[2]}),
                                      mx1, my1, mz1, mx2, my2, mz2, 
                        simplify=False, manual=True, implicit=False, 
                        warn=True, check=False, exclude=[H, tet1, tet2, A, M, w])

lin_mat_sol

{m_x1: m_z2*(-2*I*A*M*sin(theta1)/sqrt(H**2 + H*M*cos(theta1)) - I*M*sin(theta1)/sqrt(H**2 + H*M*cos(theta1))),
 m_y1: m_z2*(I*H/sqrt(H**2 + H*M*cos(theta1)) + I*M*cos(theta1)/sqrt(H**2 + H*M*cos(theta1))),
 m_z1: m_z2,
 m_x2: m_z2*(2*I*A*M*sin(theta1)/sqrt(H**2 + H*M*cos(theta1)) + I*M*sin(theta1)/sqrt(H**2 + H*M*cos(theta1))),
 m_y2: m_z2*(I*H/sqrt(H**2 + H*M*cos(theta1)) + I*M*cos(theta1)/sqrt(H**2 + H*M*cos(theta1)))}

In [25]:
for key in lin_mat_sol:
    display(lin_mat_sol[key])
    

m_z2*(-2*I*A*M*sin(theta1)/sqrt(H**2 + H*M*cos(theta1)) - I*M*sin(theta1)/sqrt(H**2 + H*M*cos(theta1)))

m_z2*(I*H/sqrt(H**2 + H*M*cos(theta1)) + I*M*cos(theta1)/sqrt(H**2 + H*M*cos(theta1)))

m_z2

m_z2*(2*I*A*M*sin(theta1)/sqrt(H**2 + H*M*cos(theta1)) + I*M*sin(theta1)/sqrt(H**2 + H*M*cos(theta1)))

m_z2*(I*H/sqrt(H**2 + H*M*cos(theta1)) + I*M*cos(theta1)/sqrt(H**2 + H*M*cos(theta1)))

In [27]:
import numpy as np
#numerical frequency calculation
mu0_num = 1.2566370621219e-6
g_num = 1.76e11
J_num = 2.7e-3
t_num = 5e-9
M_num = 1042e3
H_num = 110e-3/mu0_num
A_num = J_num/(mu0_num*t*(M_num**2))
cos_tet_num = mu0_num*H_num*M_num*t_num/(2.0*J_num)

for cur_sol in lin_mat_prot_det0_sol:
    freq_eval = cur_sol.subs({
        mu0: mu0_num,
        g: g_num,
        J: J_num,
        t: t_num,
        M: M_num,
        H: H_num,
        A: A_num,
        sym.sin(tet1)**2: 1-sym.cos(tet1)**2
    }).subs({
        sym.cos(tet1): cos_tet_num
    })
    display((freq_eval.evalf()/(2*np.pi))*1e-9)

display("default freq:")
display((g_num*mu0_num*np.sqrt(H_num*(H_num+M_num))/(2*np.pi))*1e-9)

0

-4.63554441651928

4.63554441651928

-32.4481464237982

32.4481464237982

'default freq:'

11.068377491233647

In [63]:
lin_mat

Matrix([
[ I*w/(gamma*mu0),                   0,               -M*(A + 1)*sin(theta1),                0,                   0,                     -A*M*sin(theta1)],
[               0,     I*w/(gamma*mu0), -A*M*cos(theta1) + H + M*cos(theta1),                0,                   0,                      A*M*cos(theta1)],
[ A*M*sin(theta1), A*M*cos(theta1) - H,                      I*w/(gamma*mu0),  A*M*sin(theta1),    -A*M*cos(theta1),                                    0],
[               0,                   0,                      A*M*sin(theta1),  I*w/(gamma*mu0),                   0,                M*(A + 1)*sin(theta1)],
[               0,                   0,                      A*M*cos(theta1),                0,     I*w/(gamma*mu0), -A*M*cos(theta1) + H + M*cos(theta1)],
[-A*M*sin(theta1),    -A*M*cos(theta1),                                    0, -A*M*sin(theta1), A*M*cos(theta1) - H,                      I*w/(gamma*mu0)]])

In [60]:
simplify(lin_mat_det.subs({w:lin_mat_det0_sol[4]}))

0

# Legacy

In [19]:
%%space verification

linear_mat_asym = sym.Matrix(
[
    [jw/(g*mu0),0, -M*sym.sin(tet1)+A*M*sym.sin(tet2)/2, 0, 0, -A*M*sym.sin(tet1)/2],
    [0, jw/(g*mu0), H+M*sym.cos(tet1)-A*M*sym.cos(tet2)/2,0,0,A*M*sym.cos(tet1)/2],
    [-A*M*sym.sin(tet2)/2, -H+A*M*sym.cos(tet2)/2, jw/(g*mu0), A*M*sym.sin(tet1)/2, -A*M*sym.cos(tet1)/2, 0],
    [0,0,-A*M*sym.sin(tet2),jw/(g*mu0), A*M*sym.sin(tet1)-M*sym.sin(tet2),0],
    [0,0,A*M*sym.cos(tet2),0,jw/(g*mu0),H-A*M*sym.cos(tet1)+M*sym.cos(tet2)],
    [A*M*sym.sin(tet2), -A*M*sym.cos(tet2), 0, -A*M*sym.sin(tet1), -H+A*M*sym.cos(tet1), jw/(g*mu0)]
]
)
linear_mat

Matrix([
[   I*w/(gamma*mu0),                     0,      A*M*sin(theta2)/2 - M*sin(theta1),                 0,                               0,                   -A*M*sin(theta1)/2],
[                 0,       I*w/(gamma*mu0), -A*M*cos(theta2)/2 + H + M*cos(theta1),                 0,                               0,                    A*M*cos(theta1)/2],
[-A*M*sin(theta2)/2, A*M*cos(theta2)/2 - H,                        I*w/(gamma*mu0), A*M*sin(theta1)/2,              -A*M*cos(theta1)/2,                                    0],
[                 0,                     0,                       -A*M*sin(theta2),   I*w/(gamma*mu0), A*M*sin(theta1) - M*sin(theta2),                                    0],
[                 0,                     0,                        A*M*cos(theta2),                 0,                 I*w/(gamma*mu0), -A*M*cos(theta1) + H + M*cos(theta2)],
[   A*M*sin(theta2),      -A*M*cos(theta2),                                      0,  -A*M*sin(theta1),             A

In [5]:
%%space verification

linear_mat_sym = simplify(sym.Matrix(
[
    [jw/(g*mu0),0, -M*sym.sin(tet1)+A*M*sym.sin(tet2), 0, 0, -A*M*sym.sin(tet1)],
    [0, jw/(g*mu0), H+M*sym.cos(tet1)-A*M*sym.cos(tet2),0,0,A*M*sym.cos(tet1)],
    [-A*M*sym.sin(tet2), -H+A*M*sym.cos(tet2), jw/(g*mu0), A*M*sym.sin(tet1), -A*M*sym.cos(tet1), 0],
    [0,0,-A*M*sym.sin(tet2),jw/(g*mu0), A*M*sym.sin(tet1)-M*sym.sin(tet2),0],
    [0,0,A*M*sym.cos(tet2),0,jw/(g*mu0),H-A*M*sym.cos(tet1)+M*sym.cos(tet2)],
    [A*M*sym.sin(tet2), -A*M*sym.cos(tet2), 0, -A*M*sym.sin(tet1), -H+A*M*sym.cos(tet1), jw/(g*mu0)]
]
).subs({sym.sin(tet2):-sym.sin(tet1), sym.cos(tet2): sym.cos(tet1)}))

C = -simplify((eq_mat.subs({mx1:0,my1:0,mz1:0,mx2:0,my2:0,mz2:0 }))/(g*mu0)).expand()
#C
#sym.solve_linear_system(linear_mat.col_insert(6, C), mx1, my1, mz1, mx2, my2, mz2)
determ = (linear_mat_sym.det()*(g*mu0)**6)
det_sol1 = sym.solve(simplify(sym.re(determ)), w)
det_sol2 = sym.solve(simplify(sym.im(determ)), w)

In [6]:
%%space verification
for i in range(1, len(det_sol1)):
    display(simplify(det_sol1[i]))

-sqrt(H*(H + M*cos(theta1)))*Abs(gamma*mu0)

sqrt(H*(H + M*cos(theta1)))*Abs(gamma*mu0)

-sqrt(5*A**2*M**2*cos(theta1)**2 - A**2*M**2 - 4*A*H*M*cos(theta1) - 3*A*M**2*cos(theta1)**2 + A*M**2 + H**2 + H*M*cos(theta1))*Abs(gamma*mu0)

sqrt(5*A**2*M**2*cos(theta1)**2 - A**2*M**2 - 4*A*H*M*cos(theta1) - 3*A*M**2*cos(theta1)**2 + A*M**2 + H**2 + H*M*cos(theta1))*Abs(gamma*mu0)

In [7]:
%%space verification
for i in range(1, len(det_sol2)):
    display(simplify(det_sol2[i]))

-gamma*mu0*sqrt(H*(H + M*cos(theta1)))

gamma*mu0*sqrt(H*(H + M*cos(theta1)))

In [16]:
%%space verification
sub_mat = linear_mat_sym.subs({w: det_sol2[1]})
sym.solve_linear_system(sub_mat.col_insert(6, sym.Matrix([0]*6)), mx1, my1, mz1, mx2, my2, mz2)

KeyboardInterrupt: 

In [8]:
%%space verification
determ.subs({w: g*mu0*(H**2+2*A**2*M**2+H*M*sym.cos(tet1)-4*A*H*M*sym.cos(tet1))})

2*I*A**3*H**2*M**3*gamma**6*mu0**6*(2*A**2*M**2 - 4*A*H*M*cos(theta1) + H**2 + H*M*cos(theta1))*sin(theta1)**2*cos(theta1) + 2*I*A**3*H*M**4*gamma**6*mu0**6*(2*A**2*M**2 - 4*A*H*M*cos(theta1) + H**2 + H*M*cos(theta1))*sin(theta1)**2*cos(theta1)**2 - 2*I*A**3*M**3*gamma**6*mu0**6*(2*A**2*M**2 - 4*A*H*M*cos(theta1) + H**2 + H*M*cos(theta1))**3*sin(theta1)**2*cos(theta1) - I*A**2*H**3*M**2*gamma**6*mu0**6*(2*A**2*M**2 - 4*A*H*M*cos(theta1) + H**2 + H*M*cos(theta1))*sin(theta1)**2 + A**2*H**2*M**2*gamma**6*mu0**6*(2*A**2*M**2 - 4*A*H*M*cos(theta1) + H**2 + H*M*cos(theta1))**2*sin(theta1)**2 - 4*A**2*H**2*M**2*gamma**6*mu0**6*(2*A**2*M**2 - 4*A*H*M*cos(theta1) + H**2 + H*M*cos(theta1))**2*cos(theta1)**2 + I*A**2*H*M**4*gamma**6*mu0**6*(2*A**2*M**2 - 4*A*H*M*cos(theta1) + H**2 + H*M*cos(theta1))*sin(theta1)**2*cos(theta1)**2 + A**2*H*M**3*gamma**6*mu0**6*(2*A**2*M**2 - 4*A*H*M*cos(theta1) + H**2 + H*M*cos(theta1))**2*sin(theta1)**2*cos(theta1) - 4*A**2*H*M**3*gamma**6*mu0**6*(2*A**2*M**2 - 4

In [33]:
import numpy as np
gamma = 1.76e11
mu0 = 1.2566370621219e-6
H = 110e-3/mu0
M = 1042e3
tet = np.deg2rad(0)
t=5e-9
J = 2.7e-3
ct = mu0*H*M*t/(2*J)
fa = gamma*mu0/(2.0*np.pi)*np.sqrt(H*(H+M*ct))
fa*1e-9
np.linalg.norm([4,0,0])

4.0

In [13]:
from IPython.display import display
import sympy as sym
A, tet1, tet2, J = sym.symbols('A theta1 theta2 J')
E = J*sym.cos(tet1-tet2)-A*(sym.cos(tet1)+sym.cos(tet2))

#sym.solve([E.diff(tet1), E.diff(tet2)], [sym.cos(tet1), sym.cos(tet2)])

display(E.diff(tet1))
display(E.diff(tet2))
(sym.sin(tet1) + sym.cos(tet1)).subs({sym.sin(tet1): sym.sin(tet2)})

A*sin(theta1) - J*sin(theta1 - theta2)

A*sin(theta2) + J*sin(theta1 - theta2)

sin(theta2) + cos(theta1)

In [17]:
%%space simple_precession
from IPython.display import display
import sympy as sym
from sympy import symbols, simplify
from sympy.vector import CoordSys3D, matrix_to_vector

S = CoordSys3D('S')

M = symbols("M", real=True)
mx, my, mz = symbols("m_x m_y m_z", real = True)
m = my*S.j + mz*S.k
MS = M*S.i

M_vec = MS + m

H = symbols("H", real=True)

HZ = H*S.i


mu0, g = symbols("mu0 gamma", real=True)

Heff = HZ

w = symbols("w", real=True)
jw = sym.I*w

dmdt = jw*m

eq = (dmdt + g*mu0*(M_vec.cross(Heff))).to_matrix(S)
eq
#sol = sym.solve(M, m2.to_matrix(S).row_insert(0, m1.to_matrix(S)))

sol = sym.solve((eq/(g*mu0)).expand(), [mx, my, mz])
sol

Matrix([
[                         0],
[ H*gamma*m_z*mu0 + I*m_y*w],
[-H*gamma*m_y*mu0 + I*m_z*w]])

{m_z: 0, m_y: 0}

In [19]:
import numpy as np
mu0 = 1.2566370621219e-6
gamma=1.76e11
H = 110e-3/mu0
f = (gamma*mu0)*np.sqrt(H*(H+M))/(2*np.pi)
f*1e-9

3.081239698259094

In [27]:
import sympy as sym

a, b, c, d = sym.symbols("a b c d")
expr = 2*a + a*b + 2*b + c*3 + d*b
expr.collect((a, b))

a*(b + 2) + b*(d + 2) + 3*c